In [7]:
import numpy as np
import open3d as o3d
from sklearn.cluster import MeanShift
import trimesh
from utils import trimesh_to_o3d, o3d_to_trimesh

In [8]:
def local_spacing(pcd, k=25):
    # estimate D(p) for every point
    tree = o3d.geometry.KDTreeFlann(pcd)
    D = np.empty(len(pcd.points))
    for idx, p in enumerate(pcd.points):
        _, idxs, dists = tree.search_knn_vector_3d(p, k)
        D[idx] = np.sqrt(dists[-1])        # r = max distance
    return D

def trilateral_shift(pcd, scan_id, D, alpha=1.0, sigma_factor=1.0):
    pts = np.asarray(pcd.points);  nrm = np.asarray(pcd.normals)
    new_pts = pts.copy()
    for i, (p, n, d) in enumerate(zip(pts, nrm, D)):
        r = alpha * d;  h = 2*alpha*d
        cyl_mask = np.logical_and.reduce([
            np.linalg.norm(np.cross(pts - p, n), axis=1) < r,   # radial
            np.abs((pts - p) @ n) < h/2,                        # height
            scan_id != scan_id[i]                               # other scan
        ])
        idx = np.where(cyl_mask)[0]
        if idx.size == 0: continue
        P = pts[idx]; N = nrm[idx]
        ri = np.linalg.norm(P - p - ((P - p) @ n)[:,None]*n, axis=1)
        hi = (P - p) @ n
        rho = 1./(D[idx]**2); rho_max = rho.max()
        w = np.exp(-ri**2/(2*r**2*sigma_factor**2)) \
            * np.exp(-hi**2/(2*h**2*sigma_factor**2)) \
            * np.exp(-(rho-rho_max)**2/(2*(rho_max-rho.min())**2*sigma_factor**2))
        shift = (w*hi).sum() / (w.sum() + 1e-12)
        new_pts[i] = p + shift * n
    pcd.points = o3d.utility.Vector3dVector(new_pts)
    return pcd

In [11]:
mesh1 = trimesh.load_mesh('meshes/bottle_1.ply')
mesh2 = trimesh.load_mesh('meshes/bottle_2.ply')

mesh1_o3d = trimesh_to_o3d(mesh1)
mesh2_o3d = trimesh_to_o3d(mesh2)

pcd_1 = o3d.geometry.PointCloud()
pcd_2 = o3d.geometry.PointCloud()
pcd_1.points = mesh1_o3d.vertices
pcd_2.points = mesh2_o3d.vertices

In [6]:
print(local_spacing(pcd_1))

[0.00809954 0.00878236 0.01072629 ... 0.00753484 0.00774001 0.00787874]
